# ✅ Импорты

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pickle

import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklift.models import SoloModel
from sklift.models import TwoModels
from sklift.metrics import uplift_at_k
from sklift.datasets import fetch_hillstrom

from causalml.inference.meta.slearner import BaseSClassifier as CausalSoloModel
from causalml.inference.meta.tlearner import BaseTClassifier as CausalTwoModels

from upninja.pipelines import DataTransformers
from upninja.pipelines import BasePipeline
from upninja.utils.Score import upliftComparingHist, scoreUpliftAtK
from upninja.models import findBestParams, baseModelSelection
from upninja.models import Spaces

import matplotlib.pyplot as plt
import seaborn as sns

# ✅ Uplift на Kevin Hillstrom датасете

## ⭐ Загрузка и обработка Kevin Hillstrom датасета

In [3]:
data = fetch_hillstrom()
X, y, t = data['data'], data['target'], data['treatment']
# упростим целевую группу - сократим до была рассылка/не было рассылки
t = t.map({'Womens E-Mail':1, 'Mens E-Mail':1, 'No E-Mail':0})

X_train, X_test, y_train, y_test, t_train, t_test = train_test_split(X, 
                                                                     y, t, 
                                                                     test_size=0.3, 
                                                                     random_state=42)

## ⭐ Выбор лучшей базовой модели

In [4]:
X_prepared = DataTransformers.HillstromTransformer().fit_transform(X_train)
y_prepared = y_train.copy()

In [5]:
log_reg_best = findBestParams(LogisticRegression,
                               X_prepared,
                               y_prepared,
                               Spaces.log_reg_hp_space
                              )

with open('saved_models/best_base/log_reg.pkl', 'wb') as f:
    pickle.dump(log_reg_best, f)

knn_best = findBestParams(KNeighborsClassifier,
                               X_prepared,
                               y_prepared,
                               Spaces.knn_hp_space
                              )

with open('saved_models/best_base/knn.pkl', 'wb') as f:
    pickle.dump(knn_best, f)

dt_best = findBestParams(DecisionTreeClassifier,
                               X_prepared,
                               y_prepared,
                               Spaces.dt_hp_space
                              )

with open('saved_models/best_base/dt.pkl', 'wb') as f:
    pickle.dump(dt_best, f)

rf_best = findBestParams(RandomForestClassifier,
                               X_prepared,
                               y_prepared,
                               Spaces.rf_hp_space
                              )

with open('saved_models/best_base/rf.pkl', 'wb') as f:
    pickle.dump(rf_best, f)

cb_best = findBestParams(CatBoostClassifier,
                               X_prepared,
                               y_prepared,
                               Spaces.cb_hp_space
                              )

with open('saved_models/best_base/cb.pkl', 'wb') as f:
    pickle.dump(cb_best, f)

  0%|                                     | 0/5 [00:00<?, ?trial/s, best loss=?]

/usr/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/usr/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/usr/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)


 20%|██        | 1/5 [00:04<00:18,  4.64s/trial, best loss: -0.6274522168450039]

/usr/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)


100%|██████████| 5/5 [01:04<00:00, 12.96s/trial, best loss: -0.6120419386189359]


In [6]:
with open('saved_models/best_base/log_reg.pkl', 'rb') as f:
    log_reg_best = pickle.load(f)

with open('saved_models/best_base/knn.pkl', 'rb') as f:
    knn_best = pickle.load(f)

with open('saved_models/best_base/dt.pkl', 'rb') as f:
    dt_best = pickle.load(f)

with open('saved_models/best_base/rf.pkl', 'rb') as f:
    rf_best = pickle.load(f)

with open('saved_models/best_base/cb.pkl', 'rb') as f:
    cb_best = pickle.load(f)

log_reg_best['best_params']['max_iter'] = int(log_reg_best['best_params']['max_iter'])

knn_best['best_params']['metric'] = knn_best['best_params']['metric'] if\
                                    knn_best['best_params']['metric'] in\
                                    {'pyfunc', 'l1', 'mahalanobis', 'minkowski',
                                     'braycurtis', 'l2', 'chebyshev', 'correlation',
                                     'seuclidean', 'sokalmichener', 'hamming', 'precomputed',
                                     'euclidean', 'haversine', 'cosine', 'dice', 'russellrao',
                                     'cityblock', 'sokalsneath', 'yule', 'infinity', 'sqeuclidean',
                                     'manhattan', 'nan_euclidean', 'p', 'canberra',
                                     'rogerstanimoto', 'jaccard'} else 'euclidean'
knn_best['best_params']['weights'] = knn_best['best_params']['weights'] if\
                                     knn_best['best_params']['weights'] in\
                                     {'distance', 'uniform'} else 'distance'

dt_best['best_params']['criterion'] = ['gini', 'entropy'][dt_best['best_params']['criterion']]
dt_best['best_params']['max_depth'] = int(dt_best['best_params']['max_depth'])

rf_best['best_params']['criterion'] = ['gini', 'entropy'][rf_best['best_params']['criterion']]
rf_best['best_params']['max_depth'] = int(rf_best['best_params']['max_depth'])
rf_best['best_params']['n_estimators'] = int(rf_best['best_params']['n_estimators'])

cb_best['best_params']['verbose'] = False

models = {
    'LogisticRregressionSklearn': LogisticRegression(**log_reg_best['best_params']),
    'KNearestSklearn': KNeighborsClassifier(**knn_best['best_params']),
    'TreeClassifierSklearn': DecisionTreeClassifier(**dt_best['best_params']),
    'RandomForestSklearn': RandomForestClassifier(**rf_best['best_params']),
    'GradientBoostingCatBoost': CatBoostClassifier(**cb_best['best_params'])
}

res = baseModelSelection(models, X_prepared, y_prepared)

In [7]:
res

,model_name,fit_time,score_time,test_accuracy,test_roc_auc,test_f1
1,KNearestSklearn,0.014280,1.490874,0.847366,0.553325,0.017513
2,TreeClassifierSklearn,0.040339,0.011758,0.853058,0.599789,0.000000
3,RandomForestSklearn,1.408738,0.105025,0.853058,0.616623,0.000000
0,LogisticRregressionSklearn,1.844587,0.025352,0.853058,0.627490,0.000000
4,GradientBoostingCatBoost,2.564179,0.018314,0.853036,0.613629,0.001819


## ⭐ Протестируем Scikit-uplifts

In [ ]:
s_learner_model = SoloModel(CatBoostClassifier(logging_level='Silent'))

s_learner_pipeline = BasePipeline.BasePipeline([
    ('hilstrom-transformer', DataTransformers.HillstromTransformer()),
    ('slearner', s_learner_model)
])

In [ ]:
%%time

s_learner_pipeline.fit(X_train, y_train, t_train)

In [ ]:
two_models_model = TwoModels(
    CatBoostClassifier(logging_level='Silent'),
    CatBoostClassifier(logging_level='Silent'),
    method='vanilla'
)

two_models_pipeline = BasePipeline.BasePipeline([
    ('hilstrom-transformer', DataTransformers.HillstromTransformer()),
    ('two-models', two_models_model)
])

In [ ]:
%%time

two_models_pipeline.fit(X_train, y_train, t_train)

In [ ]:
models_w_preds = (
    ('s-learner', s_learner_pipeline.predict(X_test)),
    ('two-models', two_models_pipeline.predict(X_test))
)

In [ ]:
res = upliftComparingHist(
    model_name_1=models_w_preds[0][0],
    model_predictions_1=models_w_preds[0][1],
    model_name_2=models_w_preds[1][0],
    model_predictions_2=models_w_preds[1][1]
)

In [ ]:
scoreUpliftAtK(
    models_w_preds,
    y_test,
    t_test
)

## ⭐ Протестируем Causal-ML

In [ ]:
s_learner_model = CausalSoloModel(CatBoostClassifier(logging_level='Silent'))

s_learner_pipeline = BasePipeline.BasePipeline([
    ('hilstrom-transformer', DataTransformers.HillstromTransformer()),
    ('slearner', s_learner_model)
])

In [ ]:
%%time

s_learner_pipeline.fit(X_train, y_train, t_train)

In [ ]:
two_models_model = CausalTwoModels(
    CatBoostClassifier(logging_level='Silent'),
    CatBoostClassifier(logging_level='Silent')
)

two_models_pipeline = BasePipeline.BasePipeline([
    ('hilstrom-transformer', DataTransformers.HillstromTransformer()),
    ('two-models', two_models_model)
])

In [ ]:
%%time

two_models_pipeline.fit(X_train, y_train, t_train)

In [ ]:
models_w_preds = (
    ('s-learner', s_learner_pipeline.predict(X_test).flatten()),
    ('two-models', two_models_pipeline.predict(X_test).flatten())
)

In [ ]:
res = upliftComparingHist(
    model_name_1=models_w_preds[0][0],
    model_predictions_1=models_w_preds[0][1],
    model_name_2=models_w_preds[1][0],
    model_predictions_2=models_w_preds[1][1]
)

In [ ]:
scoreUpliftAtK(
    models_w_preds,
    y_test,
    t_test
)